In [ ]:
!pip install -q pydantic transformers torch sentencepiece langchain langchain-huggingface langgraph

from pydantic import BaseModel, Field
from typing import Optional
from enum import Enum
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
from langgraph.graph import StateGraph, END
from typing import TypedDict
import operator

# **1. Data Models**

In [ ]:
class Employee(BaseModel):
    id: int
    name: str
    department: str
    remaining_vacation_days: int = Field(..., gt=0)

class VacationRequestStatus(str, Enum):
    PENDING = "Pending"
    APPROVED = "Approved"
    REJECTED = "Rejected"

class VacationRequest(BaseModel):
    employee_id: int
    start_date: str
    end_date: str
    reason: str
    status: VacationRequestStatus = VacationRequestStatus.PENDING
    comments: Optional[str] = None

# **2. State for LangGraph**

In [ ]:
class VacationRequestState(TypedDict):
    request: VacationRequest
    employee: Employee
    decision: int
    notification: str

# **3. Translation Dictionaries**

In [ ]:
persian_to_english_dept = {
    "توسعه نرم‌افزار": "Software Development",
    "منابع انسانی": "Human Resources",
    "مالی": "Finance",
    "بازاریابی": "Marketing"
}

persian_to_english_reason = {
    "شرکت در کنفرانس فناوری اطلاعات": "Attending an IT conference",
    "استراحت شخصی": "Personal rest",
    "مرخصی پزشکی": "Medical leave",
    "سفر خانوادگی": "Family trip"
}

# **4. Simple Persian-to-English Transliteration for Names**

In [ ]:
def transliterate_persian_name(persian_text: str) -> str:
    persian_to_latin = {
        'ا': 'a', 'آ': 'A', 'ب': 'b', 'پ': 'p', 'ت': 't', 'ث': 's',
        'ج': 'j', 'چ': 'ch', 'ح': 'h', 'خ': 'kh', 'د': 'd', 'ذ': 'z',
        'ر': 'r', 'ز': 'z', 'ژ': 'zh', 'س': 's', 'ش': 'sh', 'ص': 's',
        'ض': 'z', 'ط': 't', 'ظ': 'z', 'ع': 'a', 'غ': 'gh', 'ف': 'f',
        'ق': 'q', 'ک': 'k', 'گ': 'g', 'ل': 'l', 'م': 'm', 'ن': 'n',
        'و': 'v', 'ه': 'h', 'ی': 'y', ' ': ' '
    }
    latin_text = ''.join(persian_to_latin.get(char, char) for char in persian_text)
    return latin_text.capitalize()

# **5. Translation Functions**

In [ ]:
def translate_persian_to_english(text: str, mapping: dict = None) -> str:
    if mapping and text in mapping:
        return mapping[text]
    return transliterate_persian_name(text)

def translate_english_to_persian(en_notification: str, employee_name: str, decision: int) -> str:
    en_notification_lower = en_notification.lower()
    status_map = {
        1: "تایید شده",
        2: "رد شده",
        3: "نیاز به اطلاعات بیشتر"
    }

    for en_status, fa_status in zip(["approved", "rejected", "need more information"], status_map.values()):
        if en_status in en_notification_lower:
            return f"کارمند گرامی {employee_name}، درخواست مرخصی شما {fa_status}."

    fa_status = status_map.get(decision, "مشکلی در پردازش درخواست شما رخ داد")
    return f"کارمند گرامی {employee_name}، درخواست مرخصی شما {fa_status}."

# **6. Initialize Hugging Face Pipeline with smaller model and optimized settings**

In [ ]:
try:
    local_llm = pipeline(
        "text2text-generation",
        model="google/flan-t5-small",
        device="cpu",
        max_length=100,
        model_kwargs={"torch_dtype": "auto", "low_cpu_mem_usage": True}
    )
except Exception as e:
    print(f"Error loading model: {e}")
    # Fallback to smaller model if primary fails
    local_llm = pipeline(
        "text2text-generation",
        model="google/flan-t5-base",
        device="cpu",
        max_length=50
    )

# **7. Wrap the pipeline in HuggingFacePipeline with reduced max_length**

In [ ]:
llm = HuggingFacePipeline(
    pipeline=local_llm,
    model_kwargs={"temperature": 0.7, "max_length": 100}
)

# **8. Simplified Evaluation Template**

In [ ]:
evaluation_template = """
Review vacation request:
Employee: {employee_name}
Department: {department}
Dates: {start_date} to {end_date}
Reason: {reason}
Days left: {remaining_days}

Options:
1. Approve
2. Reject
3. Need more info

Respond with number only.
"""

# **9. Simplified Notification Template**

In [ ]:
notification_template = """
Notify employee about vacation request:
Name: {employee_name}
Decision: {decision}

Write a short message.
"""

# **10. Define Agent Nodes with error handling**

In [ ]:
def evaluate_request_node(state: VacationRequestState) -> VacationRequestState:
    try:
        en_employee_name = translate_persian_to_english(state["employee"].name)
        en_department = translate_persian_to_english(state["employee"].department, persian_to_english_dept)
        en_reason = translate_persian_to_english(state["request"].reason, persian_to_english_reason)

        prompt = evaluation_template.format(
            employee_name=en_employee_name,
            department=en_department,
            start_date=state["request"].start_date,
            end_date=state["request"].end_date,
            reason=en_reason,
            remaining_days=state["employee"].remaining_vacation_days
        )

        evaluation = llm.invoke(prompt)
        state["decision"] = int(evaluation.strip()[0]) if evaluation.strip() else 3
    except Exception as e:
        print(f"Evaluation error: {e}")
        state["decision"] = 3
    return state

def notify_employee_node(state: VacationRequestState) -> VacationRequestState:
    try:
        en_employee_name = translate_persian_to_english(state["employee"].name)

        prompt = notification_template.format(
            employee_name=en_employee_name,
            decision=state["decision"]
        )

        notification = llm.invoke(prompt).strip()
        if not notification:
            notification = {
                1: f"Dear {en_employee_name}, your vacation request is approved.",
                2: f"Dear {en_employee_name}, your vacation request is rejected.",
                3: f"Dear {en_employee_name}, we need more information about your request."
            }.get(state["decision"], "Request processing error")

        state["notification"] = translate_english_to_persian(notification, state["employee"].name, state["decision"])
    except Exception as e:
        print(f"Notification error: {e}")
        state["notification"] = "خطا در پردازش درخواست"
    return state

# **11. Build LangGraph Workflow**

In [ ]:
workflow = StateGraph(VacationRequestState)
workflow.add_node("evaluate_request", evaluate_request_node)
workflow.add_node("notify_employee", notify_employee_node)
workflow.add_edge("evaluate_request", "notify_employee")
workflow.add_edge("notify_employee", END)
workflow.set_entry_point("evaluate_request")
graph = workflow.compile()

# **12. Function to process vacation request**

In [ ]:
def process_vacation_request(request: VacationRequest, employee: Employee):
    initial_state = {
        "request": request,
        "employee": employee,
        "decision": 0,
        "notification": ""
    }

    try:
        result = graph.invoke(initial_state)
        return {
            "decision": result["decision"],
            "notification": result["notification"]
        }
    except Exception as e:
        print(f"Processing error: {e}")
        return {
            "decision": 3,
            "notification": "خطا در پردازش درخواست"
        }

# **13. Sample Data (in Persian)**

In [ ]:
employee = Employee(
    id=1001,
    name="امید سکاکی",
    department="توسعه نرم‌افزار",
    remaining_vacation_days=12
)

request = VacationRequest(
    employee_id=1001,
    start_date="1403/05/10",
    end_date="1403/05/12",
    reason="شرکت در کنفرانس فناوری اطلاعات",
    status=VacationRequestStatus.PENDING
)

# Process Request
print("در حال پردازش درخواست مرخصی...\n")
result = process_vacation_request(request, employee)

print("\n--- نتیجه نهایی ---")
#print(f"تصمیم: {result['decision']} (1=تایید, 2=رد, 3=نیاز به اطلاعات بیشتر)")
print(f"پیام: {result['notification']}")